In [ ]:
import sys
import os

print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

# Get the absolute path to the directory containing the current script
# For GlobalLocal/src/analysis/preproc/make_epoched_data.py, this is GlobalLocal/src/analysis/preproc
try:
    # This will work if running as a .py script
    current_file_path = os.path.abspath(__file__)
    current_script_dir = os.path.dirname(current_file_path)
except NameError:
    # This will be executed if __file__ is not defined (e.g., in a Jupyter Notebook)
    # os.getcwd() often gives the directory of the notebook,
    # or the directory from which the Jupyter server was started.
    current_script_dir = os.getcwd()

# Navigate up three levels to get to the 'GlobalLocal' directory
project_root = os.path.abspath(os.path.join(current_script_dir, '..', '..', '..'))

# Add the 'GlobalLocal' directory to sys.path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root) # insert at the beginning to prioritize it
    
from functools import partial

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne

import numpy as np
import pandas as pd
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.calc.mat import LabeledArray, combine

# TODO: hmm fix these utils imports, import the funcs individually. 6/1/25.
from src.analysis.utils.general_utils import *
from src.analysis.utils.general_utils import make_or_load_subjects_electrodes_to_ROIs_dict
import matplotlib.pyplot as plt

from pandas import read_csv
import scipy.stats as stats
import joblib

from scipy.ndimage import label
from scipy.stats import norm
from scipy.stats import ttest_ind

import json
import pickle

# rsa toolbox imports
from rsatoolbox.io.mne import read_epochs
from rsatoolbox.data.ops import merge_datasets
from rsatoolbox.rdm import calc_rdm_movie
from rsatoolbox.rdm.calc import _parse_input
from rsatoolbox.util.build_rdm import _build_rdms
from rsatoolbox.rdm import compare
from rsatoolbox.vis import show_rdm
from rsatoolbox.vis.timecourse import plot_timecourse

from os.path import join, expanduser, basename
import glob, json
import numpy, tqdm, mne, pandas
import rsatoolbox
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot

from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from ieeg.decoding.decoders import PcaLdaClassification
from ieeg.calc.oversample import MinimumNaNSplit
from ieeg.calc.fast import mixup

from src.analysis.config import experiment_conditions

from src.analysis.utils.labeled_array_utils import (
    put_data_in_labeled_array_per_roi_subject,
    remove_nans_from_labeled_array,
    remove_nans_from_all_roi_labeled_arrays,
    concatenate_conditions_by_string,
    get_data_in_time_range
)

from src.analysis.decoding.decoding import (
    process_and_balance_data_for_decoding, 
    get_and_plot_confusion_matrix_for_rois_jim,
    Decoder, 
    windower,
    get_confusion_matrices_for_rois_time_window_decoding_jim,
    compute_accuracies,
    perform_time_perm_cluster_test_for_accuracies,
    plot_accuracies
)

import mne.time_frequency
from ieeg.calc.scaling import rescale
from ieeg.timefreq.utils import wavelet_scaleogram, crop_pad
import numpy as np
from src.analysis.spec.wavelet_functions import get_uncorrected_wavelets, get_uncorrected_multitaper, get_sig_tfr_differences, plot_mask_pages

['c:\\Users\\jz421\\Desktop\\GlobalLocal', 'C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\L

#### 0. Load data.   
Need a way to load in the frequency information too, not just trials x channels x timepoints. Because I'm going to use frequency as a decoding feature too. For each electrode, for each training set, I think I can just mask the multitaper with the significant clusters and use that as the decoding feature, and then concatenate across electrodes to build the full training matrix.

In [ ]:
# subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103', 'D0107A', 'D0110', 'D0116', 'D0117', 'D0121']

# params - these will become input variables once i functionalize this stuff
subjects = ['D0057']
signal_times = [-1.0, 1.5]
acc_trials_only = True
error_trials_only = False
stat_func = partial(ttest_ind, equal_var=False)
p_thresh = 0.05
ignore_adjacency = 1 # ignore the channels dimension for clusters, just find clusters over frequency and time
n_perm = 20
n_jobs = 1
freqs = np.arange(2, 200., 2.)
n_cycles = freqs / 2
return_itc = False
time_bandwidth=10 
spec_method = 'multitaper'
# load in subjects electrodes to rois dict. If it doesn't already exist, make it and then load it.
config_dir = r'C:\Users\jz421\Desktop\GlobalLocal\src\analysis\config'
subjects_electrodestoROIs_dict = make_or_load_subjects_electrodes_to_ROIs_dict(subjects, task='GlobalLocal', LAB_root=None, save_dir=config_dir, 
                                                filename='subjects_electrodestoROIs_dict.json')
HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)

task='GlobalLocal'
conditions = experiment_conditions.stimulus_switch_type_conditions # set this to whichever conditions you're running

stimulus_locked = True  #toggle
response_locked = not stimulus_locked

if stimulus_locked:
    # epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_4.0-8.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"
    epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"
    # epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_0.0-30.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"

elif response_locked:
    # epochs_root_file = "Response_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_4.0-8.0_padLength_0.5s_stat_func_ttest_ind"
    epochs_root_file = "Response_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind"

condition_names = list(conditions.keys()) # get the condition names as a list

Attempting to load the subjects' electrodes-to-ROIs dictionary...
Loaded data from C:\Users\jz421\Desktop\GlobalLocal\src\analysis\config\subjects_electrodestoROIs_dict.json
Dictionary loaded successfully. Ready to proceed!


In [16]:
if conditions == experiment_conditions.stimulus_conditions:
    conditions_save_name = 'stimulus_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_experiment_conditions:
    conditions_save_name = 'stimulus_experiment_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_main_effect_conditions:
    conditions_save_name = 'stimulus_main_effect_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_lwpc_conditions:
    conditions_save_name = 'stimulus_lwpc_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_lwps_conditions:
    conditions_save_name = 'stimulus_lwps_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_big_letter_conditions:
    conditions_save_name = 'stimulus_big_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_small_letter_conditions:
    conditions_save_name = 'stimulus_small_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_task_conditions:
    conditions_save_name = 'stimulus_task_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_congruency_conditions:
    conditions_save_name = 'stimulus_congruency_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_switch_type_conditions:
    conditions_save_name = 'stimulus_switch_type_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'

elif conditions == experiment_conditions.response_conditions:
    conditions_save_name = 'response_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_experiment_conditions:
    conditions_save_name = 'response_experiment_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_big_letter_conditions:
    conditions_save_name = 'response_big_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_small_letter_conditions:
    conditions_save_name = 'response_small_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_task_conditions:
    conditions_save_name = 'response_task_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_congruency_conditions:
    conditions_save_name = 'response_congruency_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_switch_type_conditions:
    conditions_save_name = 'response_switch_type_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'

#### 1. For each electrode, make multitaper using all training trials, for both conditions to be compared (this can only do two conditions, can't do more rn)

In [36]:
# TODO: Turn this cell into a function and implement some way of getting just accurate trials (whether using spec["Accuracy1.0"] or adding /Accuracy1.0 to the BIDS events)
# TODO: Need to loop over the trials and use some as training set, and have some held out as a test set that isn't used to find clusters. Maybe do this in the stats step though.

def make_subjects_tfr_objects(subjects, layout, conditions, spec_method, signal_times, freqs, n_cycles, time_bandwidth, return_itc, n_jobs, acc_trials_only=False, error_trials_only=False, conditions_save_name="tfr_data"):
    '''
    Calculates and saves time-frequency representations (TFRs) for subjects.

    This function iterates through a list of subjects and experimental conditions
    to compute TFRs using either multitaper or wavelet analysis. It handles
    the selection of accurate or error trials and saves the resulting TFR
    objects for all subjects into a single .npy file.

    Note: This function assumes that the underlying data for TFR calculation
    is not averaged across trials yet, allowing for trial-level operations
    (e.g., train-test splits) at a later stage.

    Args:
        subjects (list of str): A list of subject identifiers (e.g., ['01', '02']).
        layout (bids.layout.BIDSLayout): A BIDSLayout object pointing to the dataset root.
        conditions (dict): A dictionary where keys are condition names (str) and
            values are dicts containing a list of BIDS event names under the key "BIDS_events".
            Example: {'ConditionA': {'BIDS_events': ['event/A1', 'event/A2']}}
        spec_method (str): The spectral estimation method to use. Must be either
            'multitaper' or 'wavelets'.
        signal_times (tuple of float): A tuple defining the time window for TFR
            calculation in seconds, e.g., (-0.5, 1.5).
        freqs (np.ndarray): An array of frequencies for the analysis.
        n_cycles (int or np.ndarray): The number of cycles for the wavelet transform.
        time_bandwidth (float): The time-bandwidth product for multitaper analysis.
        return_itc (bool): If True, calculates inter-trial coherence.
        n_jobs (int): The number of parallel jobs to run.
        acc_trials_only (bool, optional): If True, appends "/Accuracy1.0" to BIDS
            event names to select only accurate trials. Defaults to False.
        error_trials_only (bool, optional): If True, appends "/Accuracy0.0" to BIDS
            event names to select only error trials. Defaults to False.
        conditions_save_name (str, optional): The base name for the output file.
            Defaults to "tfr_data".
    '''
    subjects_tfr_objects_save_dir = os.path.join(layout.root, 'derivatives', 'spec', spec_method, 'subjects_tfr_objects')
    if not os.path.exists(subjects_tfr_objects_save_dir):
        os.makedirs(subjects_tfr_objects_save_dir)

    subjects_tfr_objects = {}

    # TODO: Kind of make a subjects_mne_objects thing but for wavelets/multitaper i think. Need to store data across subjects. 
    # Once I get this all working with one person, I need to run this on the cluster this is so slow. Don't have space to save the outputs locally.
    # OR I can save the outputs on box and load them in, similar to the epochs and tfr objects. This is probably best solution i think.
    for sub in subjects:
        subjects_tfr_objects[sub] = {}

        # load in good data so we can use it for gettin bad channels and getting filenames
        good = get_good_data(sub, layout)

        # make signal wavelets
        for condition_name, condition_dict in conditions.items():
            BIDS_events = condition_dict["BIDS_events"] # extract the list of BIDS events

            if acc_trials_only:
                BIDS_events = [BIDS_event + "/Accuracy1.0" for BIDS_event in BIDS_events]
            elif error_trials_only:
                BIDS_events = [BIDS_event + "/Accuracy0.0" for BIDS_event in BIDS_events]
            if spec_method == 'multitaper':
                spec = get_uncorrected_multitaper(sub, layout, BIDS_events, signal_times, freqs, n_cycles, time_bandwidth, return_itc, n_jobs)
            elif spec_method == 'wavelets':
                spec = get_uncorrected_wavelets(sub, layout, BIDS_events, signal_times, freqs, n_cycles, time_bandwidth, return_itc, n_jobs)
            fnames = [os.path.relpath(f, layout.root) for f in good.filenames]
            spec.info['subject_info']['files'] = tuple(fnames)
            spec.info['bads'] = good.info['bads']
            subjects_tfr_objects[sub][condition_name] = spec

            '''
            Ah oh good i don't average across trials yet, so i think i can just make these once for each subject and condition and then later do the train test split before doing the stats.
            I'll need to figure out how to map the specified trials from the spectra to the epochs objects from subjects_mne_objects i think.
            Oh i can do it in the same step where i cross validate. Because the epochs and spectra should theoretically be the same shape. 
            Hm maybe trial outliers though, may need to match trials..
            wait why would i need epochs objects at all? tfr should be enough, it has trials x channels x time x frequency (in some order, check the order)
            '''

    np.save(os.path.join(subjects_tfr_objects_save_dir, conditions_save_name + spec_method), subjects_tfr_objects)

In [ ]:
# untested 7/4/25
make_subjects_tfr_objects(
    subjects=subjects,
    layout=layout,
    conditions=conditions,
    spec_method=spec_method,
    signal_times=signal_times,
    freqs=freqs,
    n_cycles=n_cycles,
    time_bandwidth=time_bandwidth,
    return_itc=return_itc,
    n_jobs=n_jobs,
    acc_trials_only=acc_trials_only,
    error_trials_only=error_trials_only,
    conditions_save_name=conditions_save_name
)

Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_ieeg.edf...
EDF file detected
Setting channel info str

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_channels.tsv.


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 228 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS

...

Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-04_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-04_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 226 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


<RawEDF | sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.edf, 178 x 8243200 (4025.0 s), ~201 kB, data not loaded>
outlier round 1 channels: ['RAMT8']
outlier round 2 channels: ['RAMT8', 'RPI16']
outlier round 2 channels: ['RAMT8', 'RPI16', 'LAMT12']
Reading 0 ... 6198083  =      0.000 ...  3026.408 secs...
Applying average reference.
Applying a custom ('sEEG',) reference.
Used Annotations descriptions: ['1', '2', 'Response/c25.0/n25.0/BigLetters/SmallLetters/Taskl/TargetLetters/Responded1.0/ParticipantResponse115.0/CorrectResponse115.0/TrialCount225.0/BlockTrialCount1.0/ReactionTime2149.0/Accuracy1.0/D57', 'Response/c25.0/r25.0/BigLetterh/SmallLetterh/Taskg/TargetLetterh/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount227.0/BlockTrialCount3.0/ReactionTime883.0/Accuracy1.0/D57', 'Response/c25.0/r25.0/BigLetterh/SmallLetterh/Taskg/TargetLetterh/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount228.0/BlockTrialCount4.0/ReactionTime766.0/Ac

c:\Users\jz421\Desktop\GlobalLocal\src\analysis\utils\general_utils.py:1592: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  all_trials = mne.concatenate_epochs(all_trials_list)


Not setting metadata
212 matching events found
No baseline correction applied
NOTE: tfr_multitaper() is a legacy function. New code should use .compute_tfr(method="multitaper").


In [ ]:
# do this if loading in the tfr object instead of making it. 
# TODO: This procedure should be split into three steps. 
# 1. Check if the subjects_tfr_objects for this condition exists. 2. If so, load it. 3. If not, make it.

subjects_tfr_objects = np.load(os.path.join(subjects_tfr_objects_save_dir, conditions_save_name + '_' + spec_method + '.npy'), allow_pickle=True).item()
subjects_tfr_objects['D0057']['Stimulus_s'].get_data().shape

# uhhh why the fuck is this returning a 3d array instead of a 4d one? the trials dimension is gone...rerun make_subjects_tfr_objects, maybe I inadvertently averaged it when i was running it in the cell.


(175, 99, 255)

#### 2. Find clusters that are significantly different between the two conditions, in the multitaper spectrogram


In [ ]:
# untested
# i think this should be done for each electrode individually, and concatenate across electrodes in an roi for the plots. 
# TODO: Fix the plotting code below, it's copied from wavelet_differences.
# TODO: Filter by roi first i think before running the stats maybe. Or can run for all channels as a start, and then filter by roi. Save the plots based on roi name or 'all_channels'.
mask, pvals = get_sig_tfr_differences(spec_rescaled_condition_1, spec_rescaled_condition_2, stat_func=stat_func)

# plot

ch_names = spec_rescaled_condition_1.ch_names  # list of channel names

# Now plot the mask pages:
mask_pages = plot_mask_pages(mask,
                    ch_names,
                    times=spec_rescaled_condition_1.times,
                    freqs=spec_rescaled_condition_1.freqs,
                    channels_per_page=60,
                    grid_shape=(6, 10),
                    cmap=parula_map,
                    title_prefix=f"{sub} ",
                    log_freq=True,
                    show=False)

# Save each page as a separate figure file:
for i, fig in enumerate(mask_pages):
    if rescaled:
        fig_name = f"{sub}_sig_multitaper_clusters_{['incongruent']['output_name']}-{conditions_and_output_names_and_events['congruent']['output_name']}_rescaled_page_{i+1}.jpg"
    else:
        fig_name = f"{sub}_sig_multitaper_clusters_{conditions_and_output_names_and_events['incongruent']['output_name']}-{conditions_and_output_names_and_events['congruent']['output_name']}_uncorrected_page_{i+1}.jpg"
    fig_pathname = os.path.join(save_dir, fig_name)
    fig.savefig(fig_pathname, bbox_inches='tight')
    print("Saved figure:", fig_name)

# get the mean differences themselves and plot them
mean_diff_inc_vs_con = mean_diff(incongruent_spec._data, congruent_spec._data, axis=0)
mean_diff_switch_vs_repeat = mean_diff(switch_spec._data, repeat_spec._data, axis=0)

# Now, plot the mean differences directly:
congruency_mean_diff_pages = plot_mask_pages(
    mean_diff_inc_vs_con,
    incongruent_spec.ch_names,
    times=incongruent_spec.times,
    freqs=incongruent_spec.freqs,
    grid_shape=(6, 10),
    cmap=parula_map,  # play with color maps
    title_prefix=f"{sub} Mean Inc-Con Diff: ",
    log_freq=True,
    show=False
)

# Save each page as a separate figure file:
for i, fig in enumerate(congruency_mean_diff_pages):
    if rescaled:
        fig_name = f"{sub}_inc-con_mean_diff_multitaper_{conditions_and_output_names_and_events['incongruent']['output_name']}-{conditions_and_output_names_and_events['congruent']['output_name']}_rescaled_page_{i+1}.jpg"
    else:
        fig_name = f"{sub}_inc-con_mean_diff_multitaper_{conditions_and_output_names_and_events['incongruent']['output_name']}-{conditions_and_output_names_and_events['congruent']['output_name']}_uncorrected_page_{i+1}.jpg"
    fig_pathname = os.path.join(save_dir, fig_name)
    fig.savefig(fig_pathname, bbox_inches='tight')
    print("Saved figure:", fig_name)

#### 3. Train a decoder on just the significant time-frequency clusters, test on the test trials

#### 4. Repeat but with new test trials (cross-validate)

#### 5. Figure out a way of plotting this. This is univariate approach, make things modular so I can do multivariate later once I figure that out.